##Main file for QCP

In [4]:
from QuantumRegister import *
from grovers import *
from deutsch import * 
from deutschJozsa import * 

In [11]:
reg = QuantumRegister(8)
for i in range(reg.n_qubits_):
    reg.applyGate(gate = reg.hadamard, target = i)
print(reg.state_)



[0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j
 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j
 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j
 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j
 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j
 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j
 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j
 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j
 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j
 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j
 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j
 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j
 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j
 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j
 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625+0.j 0.0625